## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-19-19-09-03 +0000,nypost,UK blocks Trump from using RAF bases for strik...,https://nypost.com/2026/02/19/world-news/uk-bl...
1,2026-02-19-19-08-44 +0000,nyt,Bill Gates Cancels Speech in India Amid Epstei...,https://www.nytimes.com/2026/02/19/business/bi...
2,2026-02-19-19-08-26 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/19/world/...
3,2026-02-19-19-07-40 +0000,startribune,Live: Canada leads U.S. 1-0 in women’s hockey ...,https://www.startribune.com/2026-winter-olympi...
4,2026-02-19-19-07-26 +0000,nypost,Backstreet Boy Brian Littrell dealt major blow...,https://nypost.com/2026/02/19/us-news/backstre...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,36
7,iran,22
17,epstein,20
45,andrew,19
133,california,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
177,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...,131
0,2026-02-19-19-09-03 +0000,nypost,UK blocks Trump from using RAF bases for strik...,https://nypost.com/2026/02/19/world-news/uk-bl...,99
170,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...,99
103,2026-02-19-15-57-00 +0000,wsj,Former Prince Andrew Arrested as Police Probe ...,https://www.wsj.com/world/uk/former-prince-and...,97
164,2026-02-19-12-35-52 +0000,nyt,Former Prince Andrew’s Ties to Epstein Keep Sp...,https://www.nytimes.com/2026/02/19/world/europ...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
177,131,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...
170,99,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...
239,67,2026-02-19-07-49-29 +0000,nypost,Former South Korean President Yoon sentenced t...,https://nypost.com/2026/02/19/world-news/forme...
129,64,2026-02-19-14-39-21 +0000,nypost,White House warns Iran it’d be ‘very wise’ to ...,https://nypost.com/2026/02/19/us-news/white-ho...
235,61,2026-02-19-08-13-25 +0000,bbc,Bill Gates pulls out of India's AI summit over...,https://www.bbc.com/news/articles/c309qv9zglno...
370,59,2026-02-18-20-06-02 +0000,wapo,Eight skiers found dead as search continues af...,https://www.washingtonpost.com/weather/2026/02...
221,47,2026-02-19-09-58-53 +0000,nypost,Rep. Lauren Boebert’s son Tyler slapped with a...,https://nypost.com/2026/02/19/us-news/lauren-b...
208,47,2026-02-19-10-22-20 +0000,cbc,King Charles says ‘law must take its course’ a...,https://www.cbc.ca/news/livestory/andrew-mount...
252,45,2026-02-19-05-01-09 +0000,nyt,Policy Flip-Flops Hurt the British Leader. The...,https://www.nytimes.com/2026/02/19/world/europ...
155,39,2026-02-19-13-28-30 +0000,nypost,Pro-ICE ads reminding Americans of ‘dangerous ...,https://nypost.com/2026/02/19/us-news/pro-ice-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
